<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/DNN_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2023-03-23 16:11:06--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  18.9MB/s    in 0.3s    

2023-03-23 16:11:06 (18.9 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2023-03-23 16:11:06--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5052 (4.9K) [text/plain]
Saving to: ‘preprocess_sku.py’

preprocess_sku.py   100%[===================>]

In [ ]:
# ! pip install -q deepmatch
! pip install Keras-Preprocessing 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import datetime
from ast import literal_eval
# from preprocess_sku import gen_data_set, gen_model_input, gen_data_set_timesplit
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss
from keras_preprocessing.sequence import pad_sequences


time: 2.86 ms (started: 2023-03-23 19:27:23 +00:00)


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 3.11 ms (started: 2023-03-23 19:27:25 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.51 s (started: 2023-03-23 19:27:25 +00:00)


In [ ]:
import gc
gc.collect()

90

time: 2.59 s (started: 2023-03-23 19:27:30 +00:00)


GPU Configuration

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
time: 131 ms (started: 2023-03-23 16:11:57 +00:00)


In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
gpu_devices

[]

time: 10.3 ms (started: 2023-03-23 16:11:57 +00:00)


# Data

In [ ]:
data = pd.read_csv('sample_data/mik_sales_sp_10052022.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# 'sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence'
# data['hist_item_id'] = data['sku_purchase_seq'].apply(literal_eval)
# data['hist_category'] = data['most_recent_view_sku'].apply(literal_eval)
data.shape

(23206, 14)

time: 168 ms (started: 2023-02-24 15:41:38 +00:00)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23206 entries, 0 to 23205
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   data_source                         23206 non-null  object        
 1   user_id                             23206 non-null  int64         
 2   sku_number                          23206 non-null  object        
 3   qty                                 23206 non-null  int64         
 4   trans_date                          23206 non-null  datetime64[ns]
 5   created_time                        23206 non-null  datetime64[ns]
 6   email_address                       23206 non-null  object        
 7   geo_zip                             23086 non-null  object        
 8   most_frequent_device_class_general  23206 non-null  object        
 9   total_order_value                   17645 non-null  float64       
 10  total_coupon_value    

In [ ]:
data.shape, data['user_id'].nunique(), data['sku_number'].nunique()

((23206, 14), 200, 8036)

time: 6.06 ms (started: 2023-02-24 15:41:51 +00:00)


# New Data

In [ ]:
# ! pip3 install --upgrade pandas==1.4.2
# ! pip list

time: 1.71 ms (started: 2023-03-16 21:15:01 +00:00)


In [ ]:
data = pd.read_pickle('sample_data/df_input_2000_20230227.sav')
data.shape

(507896, 17)

time: 14.6 s (started: 2023-03-23 17:27:31 +00:00)


In [ ]:
data.user_id.nunique(), data.sku_number.nunique(), data.trans_date.min(), data.trans_date.max()

(1717,
 22854,
 Timestamp('2023-01-24 00:00:00'),
 Timestamp('2023-02-27 00:00:00'))

time: 351 ms (started: 2023-03-23 19:28:10 +00:00)


# Feature Engineering

In [ ]:
data.columns

Index(['user_id', 'sku_number', 'category_path', 'trans_date', 'created_time',
       'sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence',
       'seq_len', 'geo_country', 'geo_region', 'geo_city', 'geo_zipcode',
       'platform', 'label', 'sku_number_org', 'user_id_org'],
      dtype='object')

time: 12.1 ms (started: 2023-03-23 19:27:39 +00:00)


In [ ]:
sparse_features = ['sku_number', 'category_path',
                   'user_id',
                   'geo_country', 'geo_region', 'geo_city', 'geo_zipcode','platform'
                   ]

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


time: 2.94 ms (started: 2023-03-23 17:29:46 +00:00)


In [ ]:
feature_max_idx = {}
for feature in sparse_features:
    if feature == 'sku_number':
        feature_max_idx[feature] = max(data[feature].max(), data['sku_view_sequence'].explode().max()) + 1
    else:
        feature_max_idx[feature] = data[feature].max() + 1

time: 3.2 s (started: 2023-03-23 19:28:27 +00:00)


In [ ]:
user_profile = data[['user_id', 'user_id_org']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number", 'sku_number_org']].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((1717, 1), (22854, 2))

time: 394 ms (started: 2023-03-23 19:28:31 +00:00)


# Train Test Split

In [ ]:
test_start_date = data['trans_date'].max() - datetime.timedelta(7)
test_start_date

Timestamp('2023-02-20 00:00:00')

time: 6.57 ms (started: 2023-03-23 19:31:20 +00:00)


In [ ]:
feature_cols = ['user_id', 'geo_country', 'geo_region', 'geo_city', 'geo_zipcode','platform',
                'sku_number', 'category_path',
                'sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence',
                'seq_len']
train_data, train_label = data[data['trans_date'] < test_start_date][feature_cols], data[data['trans_date'] < test_start_date][['label']]
test_data, test_label = data[data['trans_date'] >= test_start_date][feature_cols], data[data['trans_date'] >= test_start_date][['label']]
train_data.shape, train_label.shape, test_data.shape, test_label.shape

((355938, 12), (355938, 1), (151958, 12), (151958, 1))

time: 366 ms (started: 2023-03-23 19:31:20 +00:00)


In [ ]:
train_data.columns

Index(['user_id', 'geo_country', 'geo_region', 'geo_city', 'geo_zipcode',
       'platform', 'sku_number', 'category_path', 'sku_purchase_seq',
       'category_path_purchase_seq', 'sku_view_sequence', 'seq_len'],
      dtype='object')

time: 4.27 ms (started: 2023-03-23 20:58:12 +00:00)


### take overlap user in test data

In [ ]:
len(set(train_data['user_id'].unique()) & set(test_data['user_id'].unique()))

848

time: 10.3 ms (started: 2023-03-23 19:42:53 +00:00)


In [ ]:
test_data = test_data[test_data['user_id'].isin(train_data['user_id'].unique())]
test_data.shape, test_data.user_id.nunique()

((118286, 12), 848)

time: 67.3 ms (started: 2023-03-23 19:42:53 +00:00)


In [ ]:
train_model_input = train_data.to_dict("list")
test_model_input = test_data.to_dict("list")

train_label = train_label.to_numpy().ravel()
test_label = test_label.to_numpy().ravel()

time: 226 ms (started: 2023-03-23 19:42:58 +00:00)


In [ ]:
for feature in ['sku_number', 'category_path','user_id','geo_country', 'geo_region', 'geo_city', 'geo_zipcode','platform',]:
  train_model_input[feature] = np.array(train_model_input[feature])
  test_model_input[feature] = np.array(test_model_input[feature])
for feature in ['sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence']:
  train_model_input[feature] = pad_sequences(train_model_input[feature], maxlen=SEQ_LEN, padding='post', truncating='post', value=0)
  test_model_input[feature] = pad_sequences(test_model_input[feature], maxlen=SEQ_LEN, padding='post', truncating='post', value=0)

time: 5.11 s (started: 2023-03-23 19:43:30 +00:00)


# Create Embeddings

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

(22854, collections.Counter, 38660)

time: 83.2 ms (started: 2023-03-23 19:44:42 +00:00)


In [ ]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

time: 3.64 ms (started: 2023-03-23 19:44:43 +00:00)


### Create Feature Embeddings



In [ ]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id',
                        'category_path',
                        'geo_country', 'geo_region', 'geo_city', 'geo_zipcode','platform',
                        'sku_purchase_seq',
                        'category_path_purchase_seq',
                        'sku_view_sequence',
                        'seq_len']

time: 2.53 ms (started: 2023-03-23 19:43:57 +00:00)


In [ ]:
user_sparse_feature_columns = ['user_id','category_path','geo_country', 'geo_region', 'geo_city', 'geo_zipcode','platform']
user_seq_sparse_feature_columns = [
    'sku_purchase_seq',
    'category_path_purchase_seq',
    'sku_view_sequence'
]

time: 2.4 ms (started: 2023-03-23 19:44:06 +00:00)


In [ ]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


time: 5.22 ms (started: 2023-03-23 19:44:46 +00:00)


In [ ]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns:
  if name[0:3] == 'sku':
    emb = Embedding(feature_max_idx['sku_number'], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
    emb.trainable = True
    sparse_embedding[name] = emb
  else:
    emb = Embedding(feature_max_idx['category_path'], embedding_dim,
                        embeddings_initializer=embeddings_initializer,
                        embeddings_regularizer=l2(
                            l2_reg),
                        name='sparse_' + 'seq_emb_'  + name,
                        mask_zero=False) # True
    emb.trainable = True
    sparse_embedding[name] = emb

time: 102 ms (started: 2023-03-23 19:46:22 +00:00)


In [ ]:
sparse_embedding

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3c70370>,
 'category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3ce28b0>,
 'geo_country': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3ce25b0>,
 'geo_region': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3ce21f0>,
 'geo_city': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8eeb0>,
 'geo_zipcode': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8e160>,
 'platform': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8e940>,
 'sku_purchase_seq': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8ed90>,
 'category_path_purchase_seq': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8e3a0>,
 'sku_view_sequence': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8ec10>}

time: 6.64 ms (started: 2023-03-23 19:46:35 +00:00)


In [ ]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

time: 3.42 ms (started: 2023-03-23 19:46:49 +00:00)


In [ ]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3c70370>,
 'category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3ce28b0>,
 'geo_country': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3ce25b0>,
 'geo_region': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3ce21f0>,
 'geo_city': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8eeb0>,
 'geo_zipcode': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8e160>,
 'platform': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8e940>,
 'sku_purchase_seq': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8ed90>,
 'category_path_purchase_seq': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8e3a0>,
 'sku_view_sequence': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3d8ec10>,
 'sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f28f3c5f940>}

time: 3.8 ms (started: 2023-03-23 19:47:05 +00:00)


# Create User Inputs


## Create Inputs Tensor

In [ ]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

time: 71.8 ms (started: 2023-03-23 19:47:12 +00:00)


In [ ]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'category_path:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_country:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_region:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_city:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_zipcode:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'platform:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'sku_purchase_seq:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'category_path_purchase_seq:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'sku_view_sequence:0' shape=(None, 50) dtype=int32>]

time: 4.89 ms (started: 2023-03-23 19:47:15 +00:00)


In [ ]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


time: 4.35 ms (started: 2023-03-23 19:47:20 +00:00)


In [ ]:
item_inputs_list

[<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>]

time: 4.9 ms (started: 2023-03-23 19:47:20 +00:00)


## Embed Inputs

In [ ]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

time: 135 ms (started: 2023-03-23 19:47:22 +00:00)


In [ ]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

time: 55.3 ms (started: 2023-03-23 19:47:23 +00:00)


In [ ]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_category_path/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_country/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_region/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_city/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_zipcode/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_platform/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_2/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

time: 7.13 ms (started: 2023-03-23 19:47:26 +00:00)


## Combine Embeded Input (User only)

In [ ]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

time: 2.46 ms (started: 2023-03-23 19:47:31 +00:00)


In [ ]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 208) dtype=float32>

time: 14.3 ms (started: 2023-03-23 19:47:32 +00:00)


# Sepecify Model Layers

### User Layers

In [ ]:
from tensorflow.keras import layers

time: 1.66 ms (started: 2023-03-23 19:47:34 +00:00)


In [ ]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

time: 1.93 ms (started: 2023-03-23 19:47:36 +00:00)


In [ ]:
# from deepctr.layers import DNN
from model_utils import DNN

time: 1.38 ms (started: 2023-03-23 19:47:36 +00:00)


In [ ]:
dnn_layers = DNN(hidden_units=user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



time: 2.43 ms (started: 2023-03-23 19:47:45 +00:00)


In [ ]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 483 ms (started: 2023-03-23 19:47:46 +00:00)


### Item Layer

In [ ]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(38660, 32) dtype=float32>

time: 846 ms (started: 2023-03-23 19:47:50 +00:00)


In [ ]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(38660, 32)

time: 539 ms (started: 2023-03-23 19:47:51 +00:00)


### Output Layer

In [ ]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

time: 180 ms (started: 2023-03-23 19:49:59 +00:00)


In [ ]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(38660, 32) dtype=float32>

time: 3.65 ms (started: 2023-03-23 19:50:07 +00:00)


In [ ]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 3.95 ms (started: 2023-03-23 19:50:10 +00:00)


In [ ]:
item_features[item_feature_name]

<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>

time: 7.45 ms (started: 2023-03-23 19:50:12 +00:00)


## Build Model

In [ ]:
from tensorflow.python.keras.models import Model

time: 1.67 ms (started: 2023-03-23 19:50:17 +00:00)


In [ ]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


time: 121 ms (started: 2023-03-23 19:50:18 +00:00)


# Model Compile and Training

In [ ]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

time: 548 ms (started: 2023-03-23 19:50:21 +00:00)


In [ ]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 355938 samples
Epoch 1/20
355938/355938 [==============================] - 93s 262us/sample - loss: 4.4615
Epoch 2/20
355938/355938 [==============================] - 90s 253us/sample - loss: 4.1196
Epoch 3/20
355938/355938 [==============================] - 89s 249us/sample - loss: 4.0455
Epoch 4/20
355938/355938 [==============================] - 89s 251us/sample - loss: 3.9950
Epoch 5/20
355938/355938 [==============================] - 91s 256us/sample - loss: 3.9773
Epoch 6/20
355938/355938 [==============================] - 89s 251us/sample - loss: 3.9588
Epoch 7/20
355938/355938 [==============================] - 89s 251us/sample - loss: 3.9523
Epoch 8/20
355938/355938 [==============================] - 89s 251us/sample - loss: 3.9511
Epoch 9/20
355938/355938 [==============================] - 89s 249us/sample - loss: 3.9441
Epoch 10/20
355938/355938 [==============================] - 89s 250us/sample - loss: 3.9405
Epoch 11/20
355938/355938 [============================

In [ ]:
import sys
sys.getsizeof(4)

28

time: 5.17 ms (started: 2023-03-23 21:11:17 +00:00)


# Predict: Generate Item and User Embeddings

In [ ]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/engine/training.py:2455: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


(118286, 32)
(22854, 32)
time: 1.46 s (started: 2023-03-23 20:25:42 +00:00)


# Recommendation

In [ ]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 45.9 MB/s eta 0:00:00
time: 7.02 s (started: 2023-03-23 20:26:07 +00:00)


In [ ]:
import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

time: 64.7 ms (started: 2023-03-23 20:26:14 +00:00)


In [ ]:
def recall_N(y_true, y_pred, N=30):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

time: 1.83 ms (started: 2023-03-23 20:26:14 +00:00)


In [ ]:
def precision_N(y_true, y_pred, N=30):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / N

time: 2.81 ms (started: 2023-03-23 20:26:14 +00:00)


In [ ]:
test_user_model_input['user_id']

array([   1,    1,    1, ..., 1716, 1716, 1716])

time: 19.8 ms (started: 2023-03-23 20:26:14 +00:00)


In [ ]:
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
  print(i,uid)
  if i >2:
    break


3it [00:00, 700.69it/s]

0 1
1 1
2 1
3 1
time: 14 ms (started: 2023-03-23 20:26:14 +00:00)


In [ ]:
test_true_label = test_data.groupby('user_id')['sku_number'].apply(list).to_dict()

time: 14.9 ms (started: 2023-03-16 21:55:47 +00:00)


In [ ]:
test_true_label = test_data.groupby('user_id')['sku_number'].apply(list).to_dict()

import numpy as np
import faiss
from tqdm import tqdm

index = faiss.IndexFlatIP(embedding_dim)
index.add(item_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
recall = []
precision = []
f1 = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['sku_number'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=30)
        precision_score = precision_N(test_true_label[uid], pred, N=30)

        recall.append(recall_score)
        precision.append(precision_score)
        f1_score = 2 * (precision_score * recall_score) /(precision_score + recall_score)
        f1.append(f1_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        pass
print("")
print("recall", np.mean(recall))
print("precision", np.mean(precision))
print("f1", np.mean(f1))
print("hit rate", hit / len(test_user_model_input['user_id']))

118286it [00:37, 3116.98it/s]


recall 0.009098230904047728
precision 0.05322382474116408
f1 0.01988723167568915
hit rate 0.0
time: 52.4 s (started: 2023-03-23 20:26:35 +00:00)


In [ ]:
print("recall", np.mean(recall)*100)
print("precision", np.mean(precision)*100)
print("f1", np.mean(f1)*100)

recall 0.9098230904047728
precision 5.322382474116408
f1 1.988723167568915
time: 46.2 ms (started: 2023-03-23 20:31:15 +00:00)


In [ ]:
item_profile['sku_number'].max()

10692

time: 15 ms (started: 2023-03-16 21:25:55 +00:00)


In [ ]:
max(list(test_true_label.values()))

10362

time: 9.05 ms (started: 2023-03-16 21:28:38 +00:00)


In [ ]:
test_user_model_input['user_id'].max()

174

time: 6.86 ms (started: 2023-03-16 21:29:39 +00:00)


In [ ]:
train_model_input['user_id'].max()

177

time: 9.08 ms (started: 2023-03-16 21:30:09 +00:00)
